In [5]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics
 
import warnings
warnings.filterwarnings('ignore')

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

stock_symbol= 'HPG'
start_date = "2018-01-01"
end_date = datetime.today()

GetFacebookInformation = yf.download(stock_symbol, start=start_date, end=end_date)
GetFacebookInformation.to_csv('GetFacebookInformation.csv')


[*********************100%***********************]  1 of 1 completed


In [9]:
import yfinance as yf

def get_pe_eps(symbol):
    try:
        # Lấy dữ liệu chứng khoán từ yfinance
        stock_data = yf.Ticker(symbol)
        
        # Lấy dữ liệu lợi nhuận net (net income) và số lượng cổ phiếu trung bình (average shares outstanding)
        net_income = stock_data.financials.loc['Net Income'].values[0]
        average_shares_outstanding = stock_data.financials.loc['Weighted Average Shs Out'].values[0]

        # Lấy giá cổ phiếu hiện tại
        current_price = stock_data.info['lastClose']

        # Tính toán chỉ số P/E và EPS
        pe_ratio = current_price / (net_income / average_shares_outstanding)
        eps = net_income / average_shares_outstanding

        # Hiển thị kết quả
        print(f"Symbol: {symbol}")
        print(f"P/E Ratio: {pe_ratio:.2f}")
        print(f"Earnings Per Share (EPS): {eps:.2f}")
    
    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu hoặc tính toán chỉ số: {str(e)}")

# Thay đổi mã chứng khoán tùy thuộc vào nhu cầu của bạn
stock_symbol = 'HAG'

# Gọi hàm để lấy và hiển thị P/E và EPS
get_pe_eps(stock_symbol)


Lỗi khi lấy dữ liệu hoặc tính toán chỉ số: yfinance failed to decrypt Yahoo data response


In [ ]:
import yfinance as yf
import csv

def get_stock_data(symbol, start_date, end_date, output_file):
    # Lấy dữ liệu chứng khoán từ yfinance
    stock_data = yf.download(symbol, start=start_date, end=end_date)

    # Lưu dữ liệu vào file CSV
    stock_data.to_csv(output_file)

    print(f'Dữ liệu chứng khoán đã được lưu vào file CSV: {output_file}')

# Thay đổi các tham số tùy thuộc vào nhu cầu của bạn
symbol = 'AAPL'  # Mã chứng khoán của Apple
start_date = '2023-01-01'
end_date = '2024-01-01'
output_file = 'stock_data.csv'

# Gọi hàm để lấy và lưu dữ liệu chứng khoán
get_stock_data(symbol, start_date, end_date, output_file)


In [3]:
import plotly.graph_objects as go
import pandas as pd


df = pd.read_csv('GetFacebookInformation.csv')

fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'])
                      ])

fig.update_layout(
    title='The Great Recession',
    yaxis_title='AAPL Stock',
    shapes = [dict(
        x0='2018-01-01', x1='2018-01-01', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations=[dict(
        x='2018-01-01', y=0.05, xref='x', yref='paper',
        showarrow=False, xanchor='left', text='Increase Period Begins')]
)

fig.show()

In [48]:
df[df['Close'] == df['Adj Close']].shape

(1037, 7)

In [49]:

df = df.drop(['Adj Close'], axis=1)

In [50]:

df.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [51]:

df['open-close']  = df['Open'] - df['Close']
df['low-high']  = df['Low'] - df['High']
df['target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

In [52]:

features = df[['open-close', 'low-high']]
target = df['target']
 
scaler = StandardScaler()
features = scaler.fit_transform(features)
 
X_train, X_valid, Y_train, Y_valid = train_test_split(
    features, target, test_size=0.1, random_state=2022)
print(X_train.shape, X_valid.shape)

(933, 2) (104, 2)


In [53]:

models = [LogisticRegression(), SVC(
  kernel='poly', probability=True), XGBClassifier()]
 
for i in range(3):
  models[i].fit(X_train, Y_train)
 
  print(f'{models[i]} : ')
  print('Training Accuracy : ', metrics.roc_auc_score(
    Y_train, models[i].predict_proba(X_train)[:,1]))
  print('Validation Accuracy : ', metrics.roc_auc_score(
    Y_valid, models[i].predict_proba(X_valid)[:,1]))
  print()

LogisticRegression() : 
Training Accuracy :  0.5353951714667459
Validation Accuracy :  0.5295668549905839

SVC(kernel='poly', probability=True) : 
Training Accuracy :  0.5135118208135392
Validation Accuracy :  0.5389830508474577

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...) : 
Training 

In [54]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import svm,neighbors
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Conv1D
from keras.callbacks import ReduceLROnPlateau

In [55]:
df['signal'] = 0.0
df['short_mavg'] = df['Close'].rolling(window=30, min_periods=1, center=False).mean()
df['long_mavg'] = df['Close'].rolling(window=120, min_periods=1, center=False).mean()
df['signal'] = np.where(df['short_mavg'] > df['long_mavg'], 1.0, 0.0)

In [56]:
def EMA(df, n):
    """ Function to calculate exponential moving average"""
    EMA = pd.Series(df['Close'].ewm(span=n, min_periods=n).mean(), name='EMA_' + str(n))
    return EMA

df['EMA21'] = EMA(df, 21)
df['EMA63'] = EMA(df, 63)
df['EMA252'] = EMA(df, 252)
df.tail()

,Date,Open,High,Low,Close,Volume,open-close,low-high,target,signal,short_mavg,long_mavg,EMA21,EMA63,EMA252
1032,2022-02-24,46750.0,46850.0,45100.0,46000.0,22445800,750.0,-1750.0,0,0.0,45055.000000,49872.916667,45717.424460,46859.500148,46722.826636
1033,2022-02-25,46100.0,46350.0,45900.0,45900.0,12775700,200.0,-450.0,1,0.0,45048.333333,49858.750000,45734.022237,46829.515768,46716.320302
1034,2022-02-28,46050.0,47800.0,46050.0,47200.0,35200100,-1150.0,-1750.0,0,0.0,45095.000000,49846.250000,45867.292943,46841.093400,46720.144892
1035,2022-03-01,47700.0,48100.0,46850.0,46900.0,23374000,800.0,-1250.0,0,0.0,45136.666667,49826.250000,45961.175402,46842.934232,46721.567053
1036,2022-03-02,47000.0,47350.0,46600.0,46900.0,19896100,100.0,-750.0,0,0.0,45196.666667,49807.083333,46046.523093,46844.717537,46722.977967


In [58]:
def ROC(df, n):
    """Function to calculate Rate of Change""" 
    M = df.diff(n - 1)  
    N = df.shift(n - 1)  
    ROC = pd.Series(((M / N) * 100), name = 'ROC_' + str(n))   
    return ROC
    
df['ROC21'] = ROC(df['Close'], 21)
df['ROC63'] = ROC(df['Close'], 63)



def MOM(df, n):
    """Function to calculate price momentum"""
    MOM = pd.Series(df.diff(n), name='Momentum_' + str(n))   
    return MOM
df['MOM21'] = MOM(df['Close'], 21)
df['MOM63'] = MOM(df['Close'], 63)



def RSI(series, period):
    """Function to calculate Relative Strength Index"""
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]

    #first value is sum of avg gains
    u[u.index[period-1]] = np.mean( u[:period] ) 
    u = u.drop(u.index[:(period-1)])

    #first value is sum of avg losses
    d[d.index[period-1]] = np.mean( d[:period] ) 
    d = d.drop(d.index[:(period-1)])
    rs = u.rolling(period-1).mean() / d.rolling(period-1).mean()
    return 100 - 100 / (1 + rs)

df['RSI21'] = RSI(df['Close'], 21)
df['RSI63'] = RSI(df['Close'], 63)
df['RSI252'] = RSI(df['Close'], 252)



#calculation of stochastic osillator.
def STOK(close, low, high, n):
    """Function to calculate Fast Stochastic Oscillator"""
    STOK = ((close - low.rolling(n).min()) / (high.rolling(n).max() - low.rolling(n).min())) * 100
    return STOK

def STOD(close, low, high, n):
    """Function to calculate Slow Stochastic Oscillator
     (3-period moving average of STOK) """
    STOK = ((close - low.rolling(n).min()) / (high.rolling(n).max() - low.rolling(n).min())) * 100
    STOD = STOK.rolling(3).mean()
    return STOD

df['%K21'] = STOK(df['Close'], df['Low'], df['High'], 21)
df['%D21'] = STOD(df['Close'], df['Low'], df['High'], 21)
df['%K63'] = STOK(df['Close'], df['Low'], df['High'], 63)
df['%D63'] = STOD(df['Close'], df['Low'], df['High'], 63)
df['%K252'] = STOK(df['Close'], df['Low'], df['High'], 252)
df['%D252'] = STOD(df['Close'], df['Low'], df['High'], 252)
df.tail()

,Date,Open,High,Low,Close,Volume,open-close,low-high,target,signal,...,MOM63,RSI21,RSI63,RSI252,%K21,%D21,%K63,%D63,%K252,%D252
1032,2022-02-24,46750.0,46850.0,45100.0,46000.0,22445800,750.0,-1750.0,0,0.0,...,-6000.0,55.952381,43.905192,51.307991,71.604938,76.131687,46.774194,46.112404,25.108225,26.695527
1033,2022-02-25,46100.0,46350.0,45900.0,45900.0,12775700,200.0,-450.0,1,0.0,...,-5500.0,58.024691,45.285215,51.281687,70.370370,74.074074,50.892857,48.724838,24.675325,25.974026
1034,2022-02-28,46050.0,47800.0,46050.0,47200.0,35200100,-1150.0,-1750.0,0,0.0,...,-2750.0,71.812081,49.054374,50.967465,86.419753,76.131687,71.794872,56.487307,30.303030,26.695527
1035,2022-03-01,47700.0,48100.0,46850.0,46900.0,23374000,800.0,-1250.0,0,0.0,...,-1100.0,64.426877,47.914184,50.787278,82.716049,79.835391,68.717949,63.801893,29.004329,27.994228
1036,2022-03-02,47000.0,47350.0,46600.0,46900.0,19896100,100.0,-750.0,0,0.0,...,-1750.0,63.855422,46.772229,50.892370,78.787879,82.641227,68.717949,69.743590,29.004329,29.437229


In [4]:
company_profile ('TCB')

NameError: name 'company_profile' is not defined